In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from sec_api import InsiderTradingApi

In [2]:
# Load .env enviroment variables
load_dotenv(dotenv_path="SEC.env")

True

In [9]:
# Set SEC API key and secret
api_key = os.getenv("SEC_API_KEY")
insiderTradingApi = InsiderTradingApi(api_key)

In [26]:
insider_trades = insiderTradingApi.get_data({
  "query": {"query_string": {"query": "issuer.tradingSymbol:TSLA"}},
  "from": "0",
  "size": "50",
  "sort": [{ "filedAt": { "order": "desc" } }]
})

In [27]:
# Assuming insider_trades contains the data you provided
insider_trades_data = insider_trades['transactions']

# Convert the data to a DataFrame
df = pd.DataFrame(insider_trades_data)

# Display the DataFrame
print(df)

                                  id           accessionNo  \
0   523ab68a6aca3f6e424f0811e26d85bb  0001790565-23-000008   
1   01b563649242f1b0400c5acc946667ba  0001972928-23-000004   
2   76cf2379be27d11c2e4f39a930beaab0  0001771340-23-000004   
3   cd48ba3bbc3b726d693e56ad9f0bdefa  0001771364-23-000007   
4   1da0b87392505d153a40b146b875b3a6  0001771340-23-000003   
5   8df8bbac7966d03c67e40699850dd1df  0001790565-23-000007   
6   d33cb647a56578903ad674f5faa6d713  0001972928-23-000003   
7   4b61713442cb60be828de1c1af7760da  0001790565-23-000006   
8   ac3d80bce521546881935e499c6e0d75  0001771340-23-000002   
9   751cb3e7d116a5f8b96d279e2d30c4eb  0001972928-23-000002   
10  88884acba0f2df891efc9ab18914155a  0001771364-23-000006   
11  3b77dba936f7de094324de6fa2f8a34b  0001790565-23-000005   
12  81e508448c02381ec866d4f22f46a606  0001972928-23-000001   
13  2979a559ab6793542ce0666810f03f39  0001771364-23-000005   
14  6d0936f6998e1fb979ef0d50c1d07a02  0001494731-23-000001   
15  6e83

In [17]:
# Assuming df is the DataFrame containing the insider trades data
df_modified = pd.DataFrame()

# Rename the 'filedAt' column to 'date'
df_modified['date'] = df['filedAt'].apply(lambda x: pd.to_datetime(x).strftime('%Y-%m-%d'))

# Extract the company name from the 'issuer' column and assign it to 'Company Name'
df_modified['Company Name'] = df['issuer'].apply(lambda x: x['name'])

# Display the modified DataFrame
df_modified.head()

,date,Company Name
0,2023-06-29,"Tesla, Inc."
1,2023-06-16,"Tesla, Inc."
2,2023-06-13,"Tesla, Inc."
3,2023-06-07,"Tesla, Inc."
4,2023-06-07,"Tesla, Inc."


In [24]:
# Assuming df is the DataFrame containing the insider trades data

# Extract 'relationship' information
df['relationship'] = df['reportingOwner'].apply(lambda x: x['relationship']['isOther'] if x and 'relationship' in x and 'isOther' in x['relationship'] else None)

# Extract 'shares' and 'pricePerShare' from 'amounts'
df['shares'] = df['nonDerivativeTable'].apply(lambda x: x['transactions'][0]['amounts']['shares'] if isinstance(x, dict) and 'transactions' in x and x['transactions'] and isinstance(x['transactions'], list) and x['transactions'][0] and 'amounts' in x['transactions'][0] and 'shares' in x['transactions'][0]['amounts'] else None)
df['pricePerShare'] = df['nonDerivativeTable'].apply(lambda x: x['transactions'][0]['amounts']['pricePerShare'] if isinstance(x, dict) and 'transactions' in x and x['transactions'] and isinstance(x['transactions'], list) and x['transactions'][0] and 'amounts' in x['transactions'][0] and 'pricePerShare' in x['transactions'][0]['amounts'] else None)

# Display the modified DataFrame
df.head()

,id,accessionNo,filedAt,schemaVersion,documentType,periodOfReport,notSubjectToSection16,issuer,reportingOwner,nonDerivativeTable,derivativeTable,footnotes,ownerSignatureName,ownerSignatureNameDate,dateOfOriginalSubmission,remarks,relationship,shares,pricePerShare
0,523ab68a6aca3f6e424f0811e26d85bb,0001790565-23-000008,2023-06-29T19:54:15-04:00,X0407,4,2023-06-27,False,"{'cik': '1318605', 'name': 'Tesla, Inc.', 'tra...","{'cik': '1790565', 'name': 'Baglino Andrew D',...",{'transactions': [{'securityTitle': 'Common St...,{'transactions': [{'securityTitle': 'Non-Quali...,"[{'id': 'F1', 'text': 'The transactions report...","By: Aaron Beckman, Power of Attorney For: Andr...",2023-06-29,NaN,NaN,False,6690.0,20.91
1,01b563649242f1b0400c5acc946667ba,0001972928-23-000004,2023-06-16T19:42:24-04:00,X0407,4,2023-06-14,False,"{'cik': '1318605', 'name': 'Tesla, Inc.', 'tra...","{'cik': '1972928', 'name': 'Zhu Xiaotong', 'ad...",{'transactions': [{'securityTitle': 'Common St...,NaN,"[{'id': 'F1', 'text': 'The transactions report...","By: Aaron Beckman, Power of Attorney For: Xiao...",2023-06-16,NaN,NaN,False,2500.0,260.00
2,76cf2379be27d11c2e4f39a930beaab0,0001771340-23-000004,2023-06-13T20:48:35-04:00,X0407,4,2023-06-09,False,"{'cik': '1318605', 'name': 'Tesla, Inc.', 'tra...","{'cik': '1771340', 'name': 'Taneja Vaibhav', '...",{'transactions': [{'securityTitle': 'Common St...,{'transactions': [{'securityTitle': 'Non-Quali...,"[{'id': 'F1', 'text': 'The transactions report...","By: Aaron Beckman, Power of Attorney For: Vaib...",2023-06-13,NaN,NaN,False,12000.0,18.22
3,cd48ba3bbc3b726d693e56ad9f0bdefa,0001771364-23-000007,2023-06-07T21:41:04-04:00,X0407,4,2023-06-05,False,"{'cik': '1318605', 'name': 'Tesla, Inc.', 'tra...","{'cik': '1771364', 'name': 'Kirkhorn Zachary',...",{'transactions': [{'securityTitle': 'Common St...,{'transactions': [{'securityTitle': 'Restricte...,"[{'id': 'F1', 'text': 'The sales reported on t...","By: Aaron Beckman, Power of Attorney For: Zach...",2023-06-07,NaN,NaN,False,3750.0,217.69
4,1da0b87392505d153a40b146b875b3a6,0001771340-23-000003,2023-06-07T21:16:32-04:00,X0407,4,2023-06-05,False,"{'cik': '1318605', 'name': 'Tesla, Inc.', 'tra...","{'cik': '1771340', 'name': 'Taneja Vaibhav', '...",{'transactions': [{'securityTitle': 'Common St...,{'transactions': [{'securityTitle': 'Restricte...,"[{'id': 'F1', 'text': 'Shares of the Issuer's ...","By: Aaron Beckman, Power of Attorney For: Vaib...",2023-06-07,NaN,NaN,False,388.0,0.00
